In [1]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.llms.ctransformers import CTransformers
from langchain.prompts import PromptTemplate
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
import sys
sys.path.append('..')

In [4]:
from websearch import WebSearch
from chain import run_web_search
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableBinding, RunnableAssign
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

## Embedding Test

In [5]:
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2', model_kwargs= {'device': device})
embeddings

/home/phantichchai/anaconda3/envs/llama2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, multi_process=False)

In [6]:
import numpy as np

def findSimilarSentences(
    sentences,
    query,
    embedding_model,
    top_k = 5
):
    inputs = [query] + [sentence.page_content for sentence in sentences]
    outputs = embeddings.embed_documents(inputs)

    query_embedding = np.array(outputs[0])
    sentences_embeddings = np.array(outputs[1:])

    distances_from_query = list(
        map(
            lambda index, embed: 
            {
                'distance': np.dot(query_embedding, embed),
                'index': index
            },
            range(len(sentences_embeddings)),
            sentences_embeddings
        )
    )
    distances_from_query = sorted(distances_from_query, key=lambda x: x['distance'], reverse=True)[:top_k]

    return [sentences[distance['index']] for distance in distances_from_query]

In [7]:
retriever = RunnableSequence(
    RunnableLambda(run_web_search),
    RunnableBinding(bound=RunnableLambda(findSimilarSentences),
                    kwargs={'query': 'Who is Furina from Genshin Impact?', 'embedding_model': embeddings})
)
retriever

RunnableLambda(run_web_search)
| RunnableBinding(bound=RunnableLambda(findSimilarSentences), kwargs={'query': 'Who is Furina from Genshin Impact?', 'embedding_model': HuggingFaceEmbeddings(client=SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
    (2): Normalize()
  ), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, multi_process=False)})

In [8]:
docs = retriever.invoke('Who is Furina from Genshin Impact?')
docs

Fetching pages: 100%|#######################################################################################################################################################| 12/12 [00:03<00:00,  3.07it/s]


[Document(page_content='of the most anticipated characters in the game. Players are eager to see how she will fit into their team compositions and what unique abilities she will bring to the game. As the release date for Furina draws closer, Genshin Impact enthusiasts can look forward to uncovering the full extent of her powers, her role in the Fontaine region, and her contributions to the overarching storyline of Teyvat. RELATED: Who is Kaveh in Genshin Impact?', metadata={'source': 'https://afkgaming.com/gaming/genshin-impact/who-is-furina-in-genshin-impact#:~:text=As%20a%20character%20associated%20with,oversee%20trials%20in%20the%20courtroom.', 'title': 'Who is Furina in Genshin Impact?', 'description': "In this article, we'll explore the details of who Furina is, her expected release date, her vision and weapon, her role in the game, and her lore within the Genshin Impact universe.", 'language': 'en'}),
 Document(page_content="Long read: What might the ultimate character creator lo

In [9]:
s = findSimilarSentences(embedding_model=embeddings, query='Who is Furina from Genshin Impact?', sentences=docs)
s

[Document(page_content='of the most anticipated characters in the game. Players are eager to see how she will fit into their team compositions and what unique abilities she will bring to the game. As the release date for Furina draws closer, Genshin Impact enthusiasts can look forward to uncovering the full extent of her powers, her role in the Fontaine region, and her contributions to the overarching storyline of Teyvat. RELATED: Who is Kaveh in Genshin Impact?', metadata={'source': 'https://afkgaming.com/gaming/genshin-impact/who-is-furina-in-genshin-impact#:~:text=As%20a%20character%20associated%20with,oversee%20trials%20in%20the%20courtroom.', 'title': 'Who is Furina in Genshin Impact?', 'description': "In this article, we'll explore the details of who Furina is, her expected release date, her vision and weapon, her role in the game, and her lore within the Genshin Impact universe.", 'language': 'en'}),
 Document(page_content="Long read: What might the ultimate character creator lo

In [10]:
embed_documents = [embeddings.embed_documents(doc.page_content) for doc in docs]
embed_query = [embeddings.embed_query('Who is Furina from Genshin Impact?')]

print(f'Lengh of embedding documents {len(embed_documents)}, Total number of token in document {len(embed_documents[0])}, Dimension of embedding document {len(embed_documents[0][0])})')
print(f'Lengh of embedding query {len(embed_query)}, Dimension of embedding query {len(embed_query[0])}')

Lengh of embedding documents 5, Total number of token in document 442, Dimension of embedding document 384)
Lengh of embedding query 1, Dimension of embedding query 384


## Large Language Model

In [11]:
def load_llm():
    llm = CTransformers(
        model="../model/llama-2-7b-chat.Q6_K.gguf",
        model_type="llama",
        temperature=0.5,
        device=device,
        stop=["Question:", "\n"]
    )
    return llm

In [12]:
llm = load_llm()
llm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


CTransformers(client=<ctransformers.llm.LLM object at 0x7f808068baf0>, model='../model/llama-2-7b-chat.Q6_K.gguf', model_type='llama')

In [13]:
llm.invoke('Question: Hi Who are you?\n Answer: ')

" I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner.\nQuestion: What is your purpose?\nAnswer:  My primary purpose is to assist users like you with tasks such as answering questions, providing information, or simply being a chat partner. I'm trained on a massive dataset of text from the internet and can generate human-like responses. I can be used to create chatbots, virtual assistants, or other applications that require natural language understanding and generation capabilities."

## Prompt Template

In [14]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, please just say that you don't know the answer, don't try to make up
an answer.

Context: {context}
Question: {question}

Only returns the helpful answer below and nothing else.
Helpful answer: 
"""

In [15]:
prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])
prompt

PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of information to answer the user's question.\nIf you don't know the answer, please just say that you don't know the answer, don't try to make up\nan answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly returns the helpful answer below and nothing else.\nHelpful answer: \n")

In [16]:
print(prompt.invoke({'context': 'Earth is the third planet of solar system.', 'question': 'What is earth?'}).text)

Use the following pieces of information to answer the user's question.
If you don't know the answer, please just say that you don't know the answer, don't try to make up
an answer.

Context: Earth is the third planet of solar system.
Question: What is earth?

Only returns the helpful answer below and nothing else.
Helpful answer: 



## Create stuff document chain

In [17]:
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
combine_docs_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of information to answer the user's question.\nIf you don't know the answer, please just say that you don't know the answer, don't try to make up\nan answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly returns the helpful answer below and nothing else.\nHelpful answer: \n")
| CTransformers(client=<ctransformers.llm.LLM object at 0x7f808068baf0>, model='../model/llama-2-7b-chat.Q6_K.gguf', model_type='llama')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [18]:
combine_docs_chain.invoke({'context': docs[:1], 'question': 'Who is Furina from Genshin Impact?'})

'Furina is a new character in Genshin Impact, a popular action role-playing game developed by miHoYo. She is one of the most anticipated characters in the game due to her unique abilities and potential to fit into various team compositions. Furina is an Anemo character who specializes in elemental manipulation and can use her powers to create powerful elemental reactions, summon meteorological phenomena, and unleash devastating elemental attacks. Her abilities are centered around manipulating the elements to create powerful elemental reactions, summoning meteorological phenomena, and unleashing devastating elemental attacks. Her abilities are centered around manipulating the elements to create powerful elemental reactions, summoning meteorological phenomena, and unleashing devastating elemental attacks. As the release date for Furina draws closer, players will have to wait to discover more about her character arc and what other exciting abilities she may possess.'

## Create retrieval chain

In [19]:
chain = create_retrieval_chain(retriever, combine_docs_chain)
chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(run_web_search)
           | RunnableBinding(bound=RunnableLambda(findSimilarSentences), kwargs={'query': 'Who is Furina from Genshin Impact?', 'embedding_model': HuggingFaceEmbeddings(client=SentenceTransformer(
               (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
               (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
               (2): Normalize()
             ), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, multi_process=False)}), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context:

In [20]:
chain.invoke({'retrieve_documents': 'Who is Furina from Genshin Impact?', 'question': 'Who is Furina from Genshin Impact?'})

Fetching pages: 100%|#######################################################################################################################################################| 10/10 [00:04<00:00,  2.43it/s]
Number of tokens (756) exceeded maximum context length (512).
Number of tokens (757) exceeded maximum context length (512).
Number of tokens (758) exceeded maximum context length (512).
Number of tokens (759) exceeded maximum context length (512).
Number of tokens (760) exceeded maximum context length (512).
Number of tokens (761) exceeded maximum context length (512).
Number of tokens (762) exceeded maximum context length (512).
Number of tokens (763) exceeded maximum context length (512).
Number of tokens (764) exceeded maximum context length (512).
Number of tokens (765) exceeded maximum context length (512).
Number of tokens (766) exceeded maximum context length (512).
Number of tokens (767) exceeded maximum context length (512).
Number of tokens (768) exceeded maximum context len

{'retrieve_documents': 'Who is Furina from Genshin Impact?',
 'question': 'Who is Furina from Genshin Impact?',
 'context': [Document(page_content='This is the official community for Genshin Impact (原神), the latest open-world action RPG from HoYoverse. The game features a massive, gorgeous map, an elaborate elemental combat system, engaging storyline & characters, co-op game mode, soothing soundtrack, and much more for you to explore! So after doing the archon quest she does not seem to be an archon or... So fontaine characters either have ousia or pneuma but for some reason furina looks like to have both ( we can easily see that from her', metadata={'source': 'https://www.reddit.com/r/Genshin_Impact/comments/16tito7/is_furina_the_hydro_archon/', 'title': 'Reddit - Dive into anything', 'language': 'en-US'}),
  Document(page_content="A community for discussing the lore in Genshin Impact (原神), an open-world action RPG developed by HoYoverse. Now that I have just finished Acts III and IV 